### Homework

- Write your own implementation of k-means algorithm with random centroid initialization and 2 stopping conditions: max iterations and centroid convergence (if all attributes of all centroids changes not more than some epsilon the algorithm should stop).
- Use your implementation to cluster data containing data about cereal products with their dietary characteristics (cereals.csv, 16 attributes).
- It contains some nominal attributes (name, mfr, type). You can omit the first two of them. Type attribute is binary, so you can replace it with values 0 and 1.
- Perform the clustering of the cereals into 3 groups using k-means algorithm.
- Remember to preprocess the given input: normalization/standarization, attribute selection.
- Try to describe the obtained groups based on the obtained centroids, what do all cereals within this group have in common?
- Write a report containing information about used preprocessing methods, number of cereals within each cluster and your conclusions about the clustering results.

## Marcin Krueger L11 145244

In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, mutual_info_classif

### Wczytywanie danych

In [145]:
def load_data(path):
    df = pd.read_csv(path)
    df = df.drop("name", axis=1)
    df = df.drop("mfr", axis=1)
    df["type"] = np.where(df["type"] == "C", 1, 0) 
    return df

df = load_data("assets/cereal.csv")
columns = df.columns
df.head()

,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,1,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,1,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,1,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,1,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,1,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


### Normalizacja wartości

Do normalizacji wartości został wykorzystany MinMaxScaler

In [146]:
data = MinMaxScaler().fit_transform(df)
data

array([[1.        , 0.18181818, 0.6       , ..., 0.5       , 0.064     ,
        0.66559279],
       [1.        , 0.63636364, 0.4       , ..., 0.5       , 0.6       ,
        0.21068456],
       [1.        , 0.18181818, 0.6       , ..., 0.5       , 0.064     ,
        0.54694061],
       ...,
       [1.        , 0.45454545, 0.4       , ..., 0.5       , 0.336     ,
        0.41955762],
       [1.        , 0.45454545, 0.4       , ..., 0.5       , 0.6       ,
        0.44341036],
       [1.        , 0.54545455, 0.2       , ..., 0.5       , 0.4       ,
        0.2398125 ]])

### Wybór atrybutów

Aby wybrać odpowiednie atrybuty wykorzystałem Variance Threshold, który pozwala się pozbyć atrybutów o zbyt małej różnorodności. Wartość progową wybrałem eksperymentalnie. Ostatecznie postanowiłem pracować na 8 atrybutach.

In [147]:
selector = VarianceThreshold(0.045)
selected = selector.fit_transform(data)

new_columns = selector.get_feature_names_out(columns)
print(f"Selected columns: {new_columns}")

deleted_columns = np.setdiff1d(np.array(columns),np.array(new_columns))
print(f"Rejected columns: {deleted_columns}")

Selected columns: ['protein' 'sodium' 'sugars' 'potass' 'vitamins' 'shelf']
Rejected columns: ['calories' 'carbo' 'cups' 'fat' 'fiber' 'rating' 'type' 'weight']


### Algorytm k-means

- Funkcja distance liczy odległość euklidesową między punktem a centroidem
- Centroidy inicjowane są jako losowa wartość ze zbioru
- Jeżeli zmiana będzie zbyt mała lub maksymalna liczba iteracji zostanie przekroczona to algorytm zakończy działanie

In [148]:
def distance(x,y):
    return np.sum((x - y) ** 2)

In [152]:
def kmeans(data, k, max_iter, eps):
    points = np.random.choice(range(len(data)), size=k)
    centroids = np.array(data[points])
    clusters = np.zeros(shape=(data.shape[0]))
    
    for _ in range(max_iter):
        for i in range(len(data)):
            min_distance = distance(data[i], centroids[0])
            clusters[i] = 0
            
            for j in range(1, len(centroids)):
                temp_distance = distance(data[i], centroids[j])
                if min_distance > temp_distance:
                    min_distance = temp_distance
                    clusters[i] = j
                    
        new_centroids = np.array([np.mean(data[clusters == i], axis=0) for i in range(len(centroids))])
        alteration_ratio = np.sum(np.abs(centroids - new_centroids))
        centroids = new_centroids
        
        if alteration_ratio < eps:
            break
    
    return centroids, clusters
            
centroids, clusters = kmeans(selected, 3, 100, 1e-6)

pd.DataFrame(centroids, columns=new_columns)

,protein,sodium,sugars,potass,vitamins,shelf
0,0.372222,0.495660,0.470486,0.395267,0.354167,1.000000
1,0.400000,0.464844,0.234375,0.246073,0.187500,0.150000
2,0.114286,0.537202,0.785714,0.163430,0.250000,0.357143


In [153]:
for i in range(len(centroids)):
    print(f"{i} centroid: {sum(clusters == i)} examples")

0 centroid: 36 examples
1 centroid: 20 examples
2 centroid: 21 examples


### Wyniki

- Klaster 0 charakteryzuje się średnią wartością 'protein', 'sugars'
- Klaster 1 charakteryzuje się wysoką wartością 'protein' oraz niską wartością 'sugars', 'vitamins' i 'shelf'
- Klaster 2 charakteryzuje się wysoką wartością 'sugars', a także niską wartością 'protein', 'potass' oraz średnią wartością 'shelf'

### Wnioski

Wyniki bardzo różnią się z każdym uruchomieniem algorytmu. Zauważyłem również, że k-means bardzo szybko trafia z centroidem do miejsca, z którego brak jest widocznej poprawy, co jest jednoznaczne z tym, że ilość iteracji jest dosyć mała.